In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM employee;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [5]:
%%sql
SELECT *
FROM track
LIMIT 5;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [10]:
%%sql
WITH combined AS (
    SELECT g.name AS genre_name, il.track_id, il.quantity
    FROM genre g
    JOIN track t
    ON g.genre_id=t.genre_id
    JOIN invoice_line il
    ON il.track_id=t.track_id
)

SELECT genre_name, SUM(quantity) AS total_quantity
FROM combined
GROUP BY genre_name;

Done.


genre_name,total_quantity
Alternative,117
Alternative & Punk,492
Blues,124
Classical,47
Drama,1
Easy Listening,74
Electronica/Dance,55
Heavy Metal,8
Hip Hop/Rap,33
Jazz,121


In [17]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 12;

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


In [ ]:
# With the analysis of sales of tracks on genres,
#I will suggest purchasing Red Tone (Punk), Slim Jim Bites (Blues) and Meteor and the Girls (Pop)

In [22]:
%%sql

WITH employee_sales AS (
    SELECT e.employee_id, e.first_name || ' ' || e.last_name AS employee_name, e.hire_date, e.title, e.city, e.country, SUM(il.unit_price * il.quantity) AS sales_amount
    FROM employee e
    JOIN customer c
    ON e.employee_id = c.support_rep_id
    JOIN invoice i
    ON c.customer_id = i.customer_id
    JOIN invoice_line il
    ON i.invoice_id = il.invoice_id
    GROUP BY e.employee_id
    ORDER BY sales_amount desc
    LIMIT 20
)

SELECT *
FROM employee_sales;

Done.


employee_id,employee_name,hire_date,title,city,country,sales_amount
3,Jane Peacock,2017-04-01 00:00:00,Sales Support Agent,Calgary,Canada,1731.5100000000143
4,Margaret Park,2017-05-03 00:00:00,Sales Support Agent,Calgary,Canada,1584.000000000013
5,Steve Johnson,2017-10-17 00:00:00,Sales Support Agent,Calgary,Canada,1393.9200000000112


In [ ]:
#There is roughly a 20% difference in amount of sales supported by Jane Peacock (the top employee) and Steve Johnson (the bottom employee)
#However it also corresponds to the difference in their hiring dates reasonably

In [32]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [ ]:
#Overall, it may worth develop the markets in Czech Republic, UK and India. However due to relative smaller number of customers data at the time of analysis, it might worth invest small money in marketing campaigns first and keep collection and analysis of future data to monitor.

In [37]:
%%sql

WITH invoice_first_track AS (
    SELECT il.invoice_id AS invoice_id, MIN(il.track_id) AS first_track_id
    FROM invoice_line il
    GROUP BY 1    
)

SELECT buying_whole_album, 
       COUNT(invoice_id) AS number_of_invoices,
       CAST(COUNT(invoice_id) AS float) / (SELECT COUNT(*) FROM invoice) AS percentage_of_invoices
       
FROM (
    SELECT ifs.*,
    CASE
        WHEN (
            SELECT t.track_id
              FROM track t
             WHERE t.album_id = (SELECT t2.album_id
                                 FROM track t2
                                 WHERE t2.track_id = ifs.first_track_id
                                 )
         
        EXCEPT 
        SELECT il2.track_id
          FROM invoice_line il2
         WHERE il2.invoice_id = ifs.invoice_id
) IS NULL
AND (
    SELECT il2.track_id
    FROM invoice_line il2
    WHERE il2.invoice_id = ifs.invoice_id
    EXCEPT 
    SELECT t.track_id
    FROM track t
    WHERE t.album_id = (
        SELECT t2.album_id
        FROM track t2
        WHERE t2.track_id = ifs.first_track_id)
) IS NULL
    THEN 'YES'
    ELSE 'NO'
    END AS Buying_whole_album
FROM invoice_first_track ifs
) subquery
GROUP BY buying_whole_album;

Done.


Buying_whole_album,number_of_invoices,percentage_of_invoices
NO,500,0.8143322475570033
YES,114,0.18566775244299674


In [ ]:
#As around 18.5% of purchases are album purchases, it is recommended for Chinook to continue buying full albums from record companies.